In [2]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/.virtualenvs/nca-alife/bin/python3


In [3]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [4]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [18]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife/bin/activate', 'cd ~/nca-alife/src'])
gpus = [0, 1, 2, 3, 4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 376 commands: 
python bptt.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/bptt/alife_0"                                  --n_iters_chunk=200 --height=32 --width=32 --dt=0.01  --p_drop=0.0 --init_state="randn" --rollout_steps=64  --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/alife.png"         --apply_loss="all"  --n_layers=1 --d_state=16 --d_embd=64 --kernel_size=3 --bs=1  --lr=0.0003 --lr_schedule="constant"    
python bptt.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/bptt/confetti_ball_0"                          --n_iters_chunk=200 --height=32 --width=32 --dt=0.01  --p_drop=0.0 --init_state="randn" --rollout_steps=64  --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/confetti_ball.png" --apply_loss="all"  --n_layers=1 --d_state=16 --d_embd=64 --kernel_size=3 --bs=1  --lr=0.0003 --lr_schedule="constant"    
python bptt.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/bptt/e

In [19]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06'

In [20]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [21]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/job_007.status
python bptt.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/bptt/wrestlers_0"                              --n_iters_chunk=200 --height=32 --width=32 --dt=0.01  --p_drop=0.0 --init_state="randn" --rollout_steps=64  --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/wrestlers.png"     --apply_loss="all"  --n_layers=1 --d_state=16 --d_embd=64 --kernel_size=3 --bs=1  --lr=0.0003 --lr_schedule="constant"     &> /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/job_007.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/job_007.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/job_015.status
python bptt.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/bptt/confetti_ball_0_dt=1.0"                   --n_

In [22]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_0.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_1.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_2.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_3.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_05_19_26_06/gpu_7.sh


# Track Experiment

In [73]:
a = []
for job_id in range(len(commands)):
    a.append(os.path.exists(f'{log_dir}/job_{job_id:03d}.out'))

In [74]:
np.array(a).mean()

1.0